In [6]:
import argparse
import copy
import os
import os.path as osp
import pprint
import sys
import time
from pathlib import Path

import open3d.ml as _ml3d
import yaml

In [7]:
basedir = "/home/threedee/repos/Open3D-ML"
datesetbase = "/home/threedee/datasets"

randlanet_smartlab_cfg = basedir + "/ml3d/configs/randlanet_smartlab.yml"
randlanet_semantickitti_cfg = basedir +"/ml3d/configs/randlanet_semantickitti.yml"
randlanet_s3dis_cfg = basedir + "/ml3d/configs/randlanet_s3dis.yml"

In [8]:
kwargs = {
    "framework": "tf",
    "device": "cuda",
    "dataset_path": datesetbase + "/SmartLab",
    "split": "training",
    "ckpt_path": None,
    "cfg_file": randlanet_smartlab_cfg,
}

args = type("args", (object,), kwargs)()

pprint.pprint(kwargs)

{'cfg_file': '/home/threedee/repos/Open3D-ML/ml3d/configs/randlanet_smartlab.yml',
 'ckpt_path': None,
 'dataset_path': '/home/threedee/datasets/SmartLab',
 'device': 'cuda',
 'framework': 'tf',
 'split': 'training'}


In [9]:
if args.framework == "torch":
    import open3d.ml.torch as ml3d
else:
    import open3d.ml.tf as ml3d
    import tensorflow as tf

def merge_cfg_file(cfg, args, extra_dict):
    if args.device is not None:
        cfg.pipeline.device = args.device
        cfg.model.device = args.device
    if args.split is not None:
        cfg.pipeline.split = args.split
    if args.dataset_path is not None:
        cfg.dataset.dataset_path = args.dataset_path
    if args.ckpt_path is not None:
        cfg.model.ckpt_path = args.ckpt_path

    return cfg.dataset, cfg.pipeline, cfg.model

In [10]:
device = args.device
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        if device == "cpu":
            tf.config.set_visible_devices([], "GPU")
        elif device == "cuda":
            tf.config.set_visible_devices(gpus[0], "GPU")
        else:
            idx = device.split(":")[1]
            tf.config.set_visible_devices(gpus[int(idx)], "GPU")
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
cfg = _ml3d.utils.Config.load_from_file(args.cfg_file)

Pipeline = _ml3d.utils.get_module("pipeline", cfg.pipeline.name, args.framework)
Model = _ml3d.utils.get_module("model", cfg.model.name, args.framework)
Dataset = _ml3d.utils.get_module("dataset", cfg.dataset.name)

cfg_dataset, cfg_pipeline, cfg_model = merge_cfg_file(cfg, args, None)

dataset = Dataset(**cfg_dataset)
model = Model(**cfg_model)
pipeline = Pipeline(model, dataset, **cfg_pipeline)

In [ ]:
# show the first 100 frames using the visualizer
vis = ml3d.vis.Visualizer()
vis.visualize_dataset(dataset, "training")  # , indices=range(100)

In [ ]:
pipeline.cfg_tb = {
    "readme": "readme",
    "cmd_line": "cmd_line",
    "dataset": pprint.pformat(cfg_dataset, indent=2),
    "model": pprint.pformat(cfg_model, indent=2),
    "pipeline": pprint.pformat(cfg_pipeline, indent=2),
}
print(pipeline.cfg_tb)

if args.split == "test":
    pipeline.run_test()
else:
    pipeline.run_train()

In [12]:
ckpt_path = "./logs/RandLANet_SmartLab_tf/checkpoint/ckpt-5"

pipeline.load_ckpt(ckpt_path=ckpt_path)

pipeline.run_test()



INFO - 2021-03-19 02:55:26,303 - semantic_segmentation - Restored from ./logs/RandLANet_SmartLab_tf/checkpoint/ckpt-5
INFO - 2021-03-19 02:55:26,317 - semantic_segmentation - Restored from ./logs/RandLANet_SmartLab_tf/checkpoint/ckpt-5
INFO - 2021-03-19 02:55:26,317 - semantic_segmentation - Logging in file : ./logs/RandLANet_SmartLab_tf/log_test_2021-03-19_02:55:26.txt
INFO - 2021-03-19 02:55:26,317 - semantic_segmentation - Started testing
INFO - 2021-03-19 02:55:26,318 - smartlab - Found 3 pointclouds for test
test:   0%|          | 0/3 [00:00<?, ?it/s]INFO - 2021-03-19 02:55:26,320 - semantic_segmentation - running inference

  0%|          | 0/20544 [00:00<?, ?it/s]

{'name': 'pcd-1615483682689850000', 'path': '/home/threedee/datasets/SmartLab/test/pcd-1615483682689850000.npy', 'split': 'test'}


WARNING - 2021-03-19 02:55:28,278 - deprecation - From %s: %s (from %s) is deprecated and will be removed %s.
Instructions for updating:
%s

100%|██████████| 20544/20544 [00:02<00:00, 8089.20it/s] 
/home/threedee/repos/Open3D-ML/ml3d/tf/modules/metrics/semseg_metric.py:51: RuntimeWarning: Mean of empty slice
  accuracies.append(np.nanmean(accuracies))
/home/threedee/repos/Open3D-ML/ml3d/tf/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))
test:  33%|███▎      | 1/3 [00:02<00:05,  2.68s/it]INFO - 2021-03-19 02:55:28,999 - semantic_segmentation - running inference

  0%|          | 0/19860 [00:00<?, ?it/s]

{'predict_labels': array([2, 2, 2, ..., 1, 1, 1]), 'predict_scores': array([[1.189e-02, 4.272e-02, 4.413e-02, 1.085e-02, 2.036e-02, 1.272e-02],
       [1.189e-02, 4.272e-02, 4.413e-02, 1.085e-02, 2.036e-02, 1.272e-02],
       [1.189e-02, 4.272e-02, 4.413e-02, 1.085e-02, 2.036e-02, 1.272e-02],
       ...,
       [5.722e-06, 1.426e-01, 2.980e-06, 4.768e-07, 0.000e+00, 5.960e-07],
       [5.722e-06, 1.426e-01, 2.980e-06, 4.768e-07, 0.000e+00, 5.960e-07],
       [5.722e-06, 1.426e-01, 2.980e-06, 4.768e-07, 0.000e+00, 5.960e-07]],
      dtype=float16)}
tf.Tensor([], shape=(0, 6), dtype=float16)
tf.Tensor([], shape=(0,), dtype=int32)
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
{'name': 'pcd-1615483693023335000', 'path': '/home/threedee/datasets/SmartLab/test/pcd-1615483693023335000.npy', 'split': 'test'}



100%|██████████| 19860/19860 [00:01<00:00, 19243.09it/s]
/home/threedee/repos/Open3D-ML/ml3d/tf/modules/metrics/semseg_metric.py:51: RuntimeWarning: Mean of empty slice
  accuracies.append(np.nanmean(accuracies))
/home/threedee/repos/Open3D-ML/ml3d/tf/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))
test:  67%|██████▋   | 2/3 [00:03<00:01,  1.77s/it]INFO - 2021-03-19 02:55:30,127 - semantic_segmentation - running inference

  0%|          | 0/20256 [00:00<?, ?it/s]

{'predict_labels': array([1, 1, 1, ..., 1, 1, 1]), 'predict_scores': array([[1.313e-02, 4.752e-02, 4.144e-02, 1.126e-02, 1.651e-02, 1.278e-02],
       [1.313e-02, 4.752e-02, 4.144e-02, 1.126e-02, 1.651e-02, 1.278e-02],
       [1.313e-02, 4.752e-02, 4.144e-02, 1.126e-02, 1.651e-02, 1.278e-02],
       ...,
       [6.139e-06, 1.426e-01, 3.040e-06, 5.364e-07, 0.000e+00, 5.960e-07],
       [6.139e-06, 1.426e-01, 3.040e-06, 5.364e-07, 0.000e+00, 5.960e-07],
       [6.139e-06, 1.426e-01, 3.040e-06, 5.364e-07, 0.000e+00, 5.960e-07]],
      dtype=float16)}
tf.Tensor([], shape=(0, 6), dtype=float16)
tf.Tensor([], shape=(0,), dtype=int32)
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]
{'name': 'pcd-1615483692023160000', 'path': '/home/threedee/datasets/SmartLab/test/pcd-1615483692023160000.npy', 'split': 'test'}



100%|██████████| 20256/20256 [00:01<00:00, 20215.99it/s]
/home/threedee/repos/Open3D-ML/ml3d/tf/modules/metrics/semseg_metric.py:51: RuntimeWarning: Mean of empty slice
  accuracies.append(np.nanmean(accuracies))
/home/threedee/repos/Open3D-ML/ml3d/tf/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))
test: 100%|██████████| 3/3 [00:04<00:00,  1.64s/it]
/home/threedee/repos/Open3D-ML/ml3d/tf/pipelines/semantic_segmentation.py:189: RuntimeWarning: Mean of empty slice
  accs = np.nanmean(np.array(accs), axis=0)
/home/threedee/repos/Open3D-ML/ml3d/tf/pipelines/semantic_segmentation.py:190: RuntimeWarning: Mean of empty slice
  ious = np.nanmean(np.array(ious), axis=0)
INFO - 2021-03-19 02:55:31,228 - semantic_segmentation - Per class Accuracy : [nan nan nan nan nan nan]
INFO - 2021-03-19 02:55:31,229 - semantic_segmentation - Per class IOUs : [nan nan nan nan nan nan]
INFO - 2021-03-19 02:55:31,229 - semantic_segmentation - Overall Acc

{'predict_labels': array([2, 2, 2, ..., 1, 1, 1]), 'predict_scores': array([[7.061e-03, 4.382e-02, 5.722e-02, 8.255e-03, 1.643e-02, 9.827e-03],
       [7.061e-03, 4.382e-02, 5.722e-02, 8.255e-03, 1.643e-02, 9.827e-03],
       [7.061e-03, 4.382e-02, 5.722e-02, 8.255e-03, 1.643e-02, 9.827e-03],
       ...,
       [7.033e-06, 1.426e-01, 3.517e-06, 5.960e-07, 5.960e-08, 7.153e-07],
       [7.033e-06, 1.426e-01, 3.517e-06, 5.960e-07, 5.960e-08, 7.153e-07],
       [7.033e-06, 1.426e-01, 3.517e-06, 5.960e-07, 5.960e-08, 7.153e-07]],
      dtype=float16)}
tf.Tensor([], shape=(0, 6), dtype=float16)
tf.Tensor([], shape=(0,), dtype=int32)
[nan, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan]


In [19]:
test_split = dataset.get_split("test")
data = test_split.get_data(0)
attr = test_split.get_attr(0)
print(attr)

results = pipeline.run_inference(data)

pprint.pprint(results['predict_labels'])
pprint.pprint(results['predict_scores'])

INFO - 2021-03-19 03:01:46,284 - smartlab - Found 3 pointclouds for test
INFO - 2021-03-19 03:01:46,285 - semantic_segmentation - running inference
  0%|          | 0/20256 [00:00<?, ?it/s]

{'name': 'pcd-1615483692023160000', 'path': '/home/threedee/datasets/SmartLab/test/pcd-1615483692023160000.npy', 'split': 'test'}


100%|██████████| 20256/20256 [00:00<00:00, 20408.72it/s]

array([1, 1, 1, ..., 1, 1, 1])
array([[1.338e-02, 4.761e-02, 4.391e-02, 1.097e-02, 1.485e-02, 1.195e-02],
       [1.338e-02, 4.761e-02, 4.391e-02, 1.097e-02, 1.485e-02, 1.195e-02],
       [1.338e-02, 4.761e-02, 4.391e-02, 1.097e-02, 1.485e-02, 1.195e-02],
       ...,
       [6.020e-06, 1.426e-01, 2.980e-06, 5.364e-07, 0.000e+00, 5.960e-07],
       [6.020e-06, 1.426e-01, 2.980e-06, 5.364e-07, 0.000e+00, 5.960e-07],
       [6.020e-06, 1.426e-01, 2.980e-06, 5.364e-07, 0.000e+00, 5.960e-07]],
      dtype=float16)
